In [2]:
import torch
import torch.nn.functional as F

In [3]:
torch.nn.MSELoss()

MSELoss()

In [4]:
torch.ones(1)

tensor([ 1.])

In [ ]:
optimizer_G = torch.optim.Adam([encoder.parameters(), decoder.parameters()])
optimizer_D = torch.optim.Adam([disc_code.parameters(), disc_img.parameters()])


# AAE phase
code_gen.train(False)
disc_img.train(False)
encoder.train()
disc_code.train()

z_code = code_gen(z)
data_repr = encoder(data)
data_rec = decoder(data_repr)
x_sampled = decoder(z_code)

loss_rec = torch.nn.MSELoss(disc_img.fmap(data)-disc_img.fmap(data))
disc_code_input = torch.cat((z_code, data_repr))
disc_code_pred = disc_code(disc_code_input)
disc_code_target = torch.cat((torch.ones(batch_size), torch.zeros(batch_size)))
gan_loss_code = torch.nn.CrossEntropyLoss(disc_code_pred, disc_code_target)
ae_loss = loss_rec - gan_loss_code


# Prior improvement phase
encoder.train(False)
disc_code.train(False)
code_gen.train()
disc_img.train()

disc_img_input = torch.cat((z_code, data_repr))
disc_img_pred = disc_img(disc_code_input)
disc_img_target = torch.cat((torch.ones(batch_size), torch.zeros(batch_size), torch.ones(batch_size)))
gan_loss_img = torch.nn.CrossEntropyLoss(disc_img_pred, disc_img_target) 


def train_epoch():
    total_loss = 0
    total_metrics = np.zeros(len(self.metrics))
    for batch_idx, (data, target) in enumerate(self.data_loader):
        data, target = data.to(self.device), target.to(self.device)

        self.optimizer.zero_grad()
        output = self.model(data)
        loss = self.loss(output, target)
        loss.backward()
        self.optimizer.step()

        total_loss += loss.item()